In [4]:
import pandas as pd

YEARS = [2020]

data = pd.DataFrame()

#ingest
for year in YEARS:
    data = data.append(pd.read_csv(f"data/play_by_play_cleaned_{year}.csv.gz", compression='gzip', low_memory=False).reset_index(drop=True), sort=True)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 300)
data.reset_index(drop=True, inplace=True)

In [5]:
# filter data for game script
DOWN_UPPER = 2
NEUTRAL = True 
if NEUTRAL:
    neutral = data.loc[ (data.play_type!="no_play") & (data.qtr<5) &
                           (data.down<=DOWN_UPPER) & (data.half_seconds_remaining>120) &
                                     (data.wp>=.2) & (data.wp<=.8)].reset_index(drop=True)
else:
     neutral = data.loc[ (data.play_type!="no_play") & (data.qtr<5)].reset_index(drop=True)

In [12]:
import numpy as np
def epa_success(epa):
    return 1 if epa>0 else 0

qtrData = neutral.groupby(['season', 'posteam']).agg(
                play_count = ('play_id', 'count'),
                pass_rate = ('pass', 'mean'),
                total_epa = ('epa', 'sum'),
                avg_epa = ('epa', 'mean'),
                epa_succ_rate = ('epa', lambda x: (x.map(epa_success).mean()))
            ).round(2).reset_index()
qtrData

,season,posteam,play_count,pass_rate,total_epa,avg_epa,epa_succ_rate
0,2020,ARI,58,0.60,-1.47,-0.03,0.43
1,2020,ATL,25,0.44,-0.02,-0.00,0.44
2,2020,BAL,19,0.42,11.51,0.61,0.63
3,2020,BUF,13,0.62,4.58,0.35,0.69
4,2020,CAR,35,0.49,0.61,0.02,0.34
5,2020,CHI,29,0.41,8.39,0.29,0.62
6,2020,CIN,43,0.51,-12.45,-0.29,0.40
7,2020,CLE,23,0.57,3.16,0.14,0.48
8,2020,DAL,42,0.57,6.62,0.16,0.48
9,2020,DET,22,0.59,-2.01,-0.09,0.36


In [11]:
import os
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
def getImage(path): 
    return OffsetImage(plt.imread(path), zoom=.5)
COLORS = {'ARI':'#97233F','ATL':'#A71930','BAL':'#241773','BUF':'#00338D','CAR':'#0085CA','CHI':'#00143F',
          'CIN':'#FB4F14','CLE':'#FB4F14','DAL':'#B0B7BC','DEN':'#002244','DET':'#046EB4','GB':'#24423C',
          'HOU':'#C9243F','IND':'#003D79','JAX':'#136677','KC':'#CA2430','LA':'#002147','LAC':'#2072BA',
          'LV':'#C4C9CC','MIA':'#0091A0','MIN':'#4F2E84','NE':'#0A2342','NO':'#A08A58','NYG':'#192E6C',
          'NYJ':'#203731','PHI':'#014A53','PIT':'#FFC20E','SEA':'#7AC142','SF':'#C9243F','TB':'#D40909',
          'TEN':'#4095D1','WAS':'#FFC20F'}

logos = os.listdir(os.getcwd() + '/logos')
logo_paths = []
for i in logos:
    logo_paths.append(os.getcwd() + '/logos/' + str(i))
QUARTERS = [1, 2, 3, 4]
for QUARTER in QUARTERS:
    qtr = qtrData.loc[qtrData.qtr==QUARTER].set_index('posteam')

    qtr['color'] = COLORS.values()
    qtr['path'] = logo_paths
    qtr.sort_values('pass_rate', ascending=False, inplace=True)

    fig, ax = plt.subplots(figsize=(30,10))

    #Adding logos to the chart
    for x0, y0, path in zip(np.arange(0,32),qtr['pass_rate']+.005,qtr['path']):
        ab = AnnotationBbox(getImage(path), (x0, y0), frameon=False, fontsize=4)
        ax.add_artist(ab)

    ax.bar(np.arange(0,32),qtr['pass_rate'],color=qtr.color,width=.5)

    nflavg = qtr['pass_rate'].mean()
    ax.axhline(y=nflavg,linestyle='--',color='black')
    ax.grid(zorder=0,alpha=.6,axis='y')
    ax.set_axisbelow(True)

    ax.set_xticks(np.arange(0,32))
    ax.set_xticklabels(qtr.index,fontsize=16)

    ax.set_yticklabels([0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7],fontsize=16)

    ax.set_ylabel('Pass Rate',fontsize=20,labelpad=20)
    title = f'Neutral Situation Pass Rates - {YEARS} \n Down 1-{DOWN_UPPER}, Win Prob. Between 20-80%, Final Two Minutes Excluded' if NEUTRAL else f'Pass Rates - {YEARS} Q{QUARTER} \n All Non-special plays'
    ax.set_title(title, fontsize=26,pad=20)
    plt.figtext(.7,.85,'Author: @uditranasaria, Data: @nflfastR, Guide: @DeryckG_',fontsize=14)

    #League average line label
    plt.text(31,nflavg+.01,'NFL Average',fontsize=14)
    plt.savefig(f'./saved/Pass_rate bar graphs/LOW_RES_{title}.png', dpi=200)
    plt.show()

ValueError: Length of values does not match length of index